In [1]:
import pandas as pd
import re
import numpy as np

def column_name(df):
    # 첫 행 살리기
    first = list (df.columns)
    if first[0] == 0:
        pass
    else:
        df.loc[0] = first
        for val in first:
            if 'Unnamed' in val:
                x = first.index (val)
                first[x] = np.nan
        df.loc[0] = first
    df = df.fillna ('')
    sem_rgx = re.compile (r'[Q][A-Z]{3}')  # semantic tagset
    syn_rgx = re.compile (r'[Y][A-Z]{3}')  # syntactic tagset
    dom_rgx = re.compile (r'[X]{1}[ABCDEFGHIJKLMNOPQRSTUVWYZ]{3}')  # domain tagset
    ent_rgx = re.compile (r'[X]{2}[A-Z]{2}')  # entity tagset
    mor_rgx = re.compile (r'[A-Z]{3}')  # morph tagset

    # 컬럼의 총 개수를 l에 저장한다.
    # 컬럼의 개수 만큼 lemma와 category뒤에 lemma와 category개수인 2를 뺀만큼
    # ''를 추가해 주어 해당 컬럼 개수 만큼의 리스트 col_nme을 만들어 준다.
    l = len (df.columns)
    col_nme = ['Lemma', 'Category']
    for i in range (l - 2):
        col_nme.append ('')
    # sem =>SemInfo 뒤에 붙을 숫자
    # syn =>SynInfo 뒤에 붙을 숫자
    # dom =>DomInfo 뒤에 붙을 숫자
    # ent =>EntInfo 뒤에 붙을 숫자
    # mor =>MorInfo 뒤에 붙을 숫자
    sem = 1
    syn = 1
    dom = 1
    ent = 1
    mor = 1

    # x를 컬럼의 개수 만큼의 숫자로 지정해 준다.
    # col_val은 해당 df의 열을 리스트화 시켜준 것이다.
    for x in range (0, l):
        col_val = df.iloc[:, x].tolist ()
        # cnt가 0이면 일치하는 값을 못 찾았다는 의미로 해석(ex 모두 빈칸인 열을 만났을 때)
        # 밑에서 cnt == 0 일때 앞에 정보를 보고 빈칸의 정보를 수정할 때 사용한다.
        cnt = 0
        # k로 col_val의 리스트 요소들을 하나씩 지정해주면서
        # k가 sem_rgx, syn_rgx, dom_rgx, ent_rgx, mor_rgx에 해당되면
        # 컬럼에 일치하는 값이 있었다는 의미로 cnt를 1 증가시켜 주고
        # Info뒤에 붙을 숫자를 1씩 증가시켜 주고
        # 비효율적인 탐색을 막기 위해 바로 break시켜준다.
        for k in col_val:
            if sem_rgx.match (k):
                col_nme[x] = 'SemInfo' + str (sem)
                sem += 1
                cnt += 1
                break

            elif syn_rgx.match (k):
                col_nme[x] = 'SynInfo' + str (syn)
                syn += 1
                cnt += 1
                break

            elif dom_rgx.match (k):
                col_nme[x] = 'DomInfo' + str (dom)
                dom += 1
                cnt += 1
                break

            elif ent_rgx.match (k):
                col_nme[x] = 'EntInfo' + str (ent)
                ent += 1
                cnt += 1
                break

            elif mor_rgx.match (k):
                col_nme[x] = 'MorInfo' + str (mor)
                mor += 1
                cnt += 1
                break

        # 만약 위에서 일치하는 값을 못찾았을 때(ex 모두 빈칸인 열이었을 때)
        # cnt는 0이므로 앞에 col_nme의 정보를 보고
        # 해당 정보와 일치하는 정보의 Info숫자를 증가시켜준 값을 해당 리스트 위치에 저장해준다.
        if cnt == 0:
            if 'Sem' in col_nme[x - 1]:
                col_nme[x] = 'SemInfo' + str (sem)
                sem += 1

            elif 'Syn' in col_nme[x - 1]:
                col_nme[x] = 'SynInfo' + str (syn)
                syn += 1

            elif 'Dom' in col_nme[x - 1]:
                col_nme[x] = 'DomInfo' + str (dom)
                dom += 1

            elif 'Ent' in col_nme[x - 1]:
                col_nme[x] = 'EntInfo' + str (ent)
                ent += 1

            elif 'Mor' in col_nme[x - 1]:
                col_nme[x] = 'MorInfo' + str (mor)
                mor += 1

    df.columns = col_nme

    return df

In [2]:
def check_info(word):
    sem_rgx = re.compile (r'[Q][A-Z]{3}')  # semantic tagset
    syn_rgx = re.compile (r'[Y][A-Z]{3}')  # syntactic tagset
    dom_rgx = re.compile (r'[X]{1}[ABCDEFGHIJKLMNOPQRSTUVWYZ]{3}')  # domain tagset
    ent_rgx = re.compile (r'[X]{2}[A-Z]{2}')  # entity tagset
    mor_rgx = re.compile (r'[A-Z]{3}')  # morph tagset

    if sem_rgx.match (word):
        return 'SenInfo'
    elif syn_rgx.match (word):
        return 'SynInfo'
    elif dom_rgx.match (word):
        return 'DomInfo'
    elif ent_rgx.match (word):
        return 'EntInfo'
    elif mor_rgx.match (word):
        return 'MorInfo'

In [4]:
merge_fname=([r"E:\Develop\python\NLP\DecoLexO\DecoLexO\example\Merge2.csv", r"E:\Develop\python\NLP\DecoLexO\DecoLexO\example\Merge3.csv"], 'CSV Files(*.csv)')

# def merge(self):

# 들어오는 데이터를 병합할 변수
merge_data = pd.read_csv (merge_fname[0][0], header=None, encoding='utf-8-sig')
merge_data = column_name (merge_data)
print("origin", merge_data)
data_files = merge_fname[0][1:]
data_df_list = []
for i in data_files:
    temp_data = pd.read_csv (i, header=None, encoding='utf-8-sig')
    temp_data = column_name (temp_data)
    data_df_list.append (temp_data)
# 두 번째부터는 들어오는 데이터를 data변수에 저장하고
# 이전에 저장해둔 merge_data와 data를 concat으로 병합한 뒤
# 두 데이터를 sort_values로 정렬을 시켜준다.
for data in data_df_list:
    print('data\n', data)
    merge_data = pd.concat ([merge_data, data], ignore_index=True)
merge_data = merge_data.sort_values (by='Lemma')
merge_data = merge_data.reset_index ()
merge_data = merge_data.fillna ('')
del merge_data['index']
print("merge_data\n", merge_data)
cnt = 1

# 우선시 되는 데이터 : my_data, 지워져도 되는 데이터 del_data
# gui상에서 .text()로 입력받는다.
rule_data = pd.read_csv (r"E:\Develop\python\NLP\DecoLexO\DecoLexO\example\Rules.csv", header=None, encoding='utf-8-sig')
rules = []
for rule_cnt in range (len (rule_data)):
    rules.append (rule_data.loc[rule_cnt].to_list ())
print("rules\n", rules)
# 사용자가 입력할 col_name.
# gui상에서 .text()로 입력받는다.
col_data = rules[0][0]

del rules[0]
print(rules)
del_row_list = []
for rule in rules:
    my_data = rule[0]
    del_data = rule[1]
    for i in range (0, len (merge_data) - 1):
        for j in range (i + 1, i + cnt + 1):
            # first에는 i번째 단어를 음절별로 나누어서 저장하고
            # second에는 j번째(i다음 단어)를 음절로 나누어서 저장한다.
            first = merge_data.loc[i, 'Lemma']
            second = merge_data.loc[j, 'Lemma']
            first_len = len (merge_data.loc[i, 'Lemma'])
            second_len = len (merge_data.loc[j, 'Lemma'])
            if first_len != second_len:
                break
                # 만약 단어의 앞글자가 같다면 아래 코드를 실행한다.
            else:
                # 앞글자는 같지만 단어가 다르면 for문을 종료한다.
                if first != second:
                    break

                # 앞글자가 같고 단어까지 같으면 입력 받은 mydata가 i번째에 있는지 아님 j(i+1)번째 있는지 확인한다.
                else:
                    for col_nms in merge_data.columns.tolist():
                        if col_data in col_nms:
                            # stem_data가 유지해야하는 data일때
                            if my_data == merge_data.loc[i, col_nms] and del_data == merge_data.loc[j, col_nms]:
                                stem_data1 = merge_data.iloc[i].values.tolist ()[2:]
                                follow_data1 = merge_data.iloc[j].values.tolist ()[2:]
                                print(stem_data1)
                                for data in stem_data1:
                                    if data not in follow_data1:
                                        col = check_info (data)
                                        col_nme = merge_data.columns.tolist()
                                        for mer_col in col_nme:
                                            if col in mer_col and merge_data.loc[j, mer_col] == del_data:
                                                merge_data.loc[j, mer_col] = data
                                                break
                                del_row_list.append (j)

                            elif my_data == merge_data.loc[j, col_nms] and del_data == merge_data.loc[i, col_nms]:
                                stem_data2 = merge_data.iloc[j].values.tolist ()[2:]
                                follow_data2 = merge_data.iloc[i].values.tolist ()[2:]
                                for data in stem_data2:
                                    if data not in follow_data2:
                                        col = check_info (data)
                                        #print("col", col)
                                        col_nme = merge_data.columns
                                        #print(col_nme)
                                        for mer_col in col_nme:
                                            if col in mer_col and merge_data.loc[i, mer_col] == del_data:
                                                merge_data.loc[i, mer_col] = data
                                                break
                                del_row_list.append (i)
                            else:
                                pass
del_temp = []
for i in del_row_list:
    del_temp.append (merge_data.iloc[i].tolist ())
del_df = pd.DataFrame (del_temp)
if len(del_df) != 0:
    del_df = column_name (del_df)
else:
    pass

print(del_df)

merge_data = merge_data.drop (del_row_list, 0)
merge_data = merge_data.reset_index ()
del merge_data["index"]

print(merge_data)

origin     Lemma Category MorInfo1
0    가나다순     NS02      ZNZ
1     개인별     NS03      ZNZ
2      개중     NS02      ZNZ
3     갱스터     NS01      ZNZ
4     검식관     NS02      ZNZ
..    ...      ...      ...
371    중장     NS02      ZNZ
372   증후군     NS02      ZNE
373   지상전     NS02      ZNZ
374   지아이     NS01      ZNZ
375   지은이     NS01      ZNE

[376 rows x 3 columns]
data
    Lemma Category MorInfo1 MorInfo2 MorInfo3 MorInfo4 MorInfo5
0    종생부     NS01      ZNW                                    
1     종언     NS02      ZNW                                    
2    종이학     NS02      ZNW                                    
3    주술적     NS02      ZNW                                    
4    주의적     NS02      ZNW                                    
5     주종     NS02      ZNW                                    
6   주체사관     NS02      ZNW                                    
7     주총     NS02      ZNW                                    
8    주춧돌     NS03      ZNW                                  